In [1]:
import torch
import numpy as np
# from models import FlowNet2
from utils.frame_utils import read_gen
import argparse
import cv2
import os
import time
from pathlib import Path
from utils import tools
from sklearn import preprocessing
import pandas as pd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim
from tensorboardX import SummaryWriter

In [2]:
class argparse():
    def __init__(self):
        super(argparse, self).__init__()
        self.rgb_max = 255.
        self.fp16 = False
        self.inference_size = [1080, 1920]
        self.crop_size = [384, 512]
        self.effective_batch_size = 1
args = argparse()

In [3]:
import torch.nn as nn
from torch.nn import init

import math
import numpy as np

try:
    from networks.resample2d_package.resample2d import Resample2d
    from networks.channelnorm_package.channelnorm import ChannelNorm

    from networks import FlowNetC
    from networks import FlowNetS
    from networks import FlowNetSD
    from networks import FlowNetFusion

    from networks.submodules import *
except:
    from .networks.resample2d_package.resample2d import Resample2d
    from .networks.channelnorm_package.channelnorm import ChannelNorm

    from .networks import FlowNetC
    from .networks import FlowNetS
    from .networks import FlowNetSD
    from .networks import FlowNetFusion

    from .networks.submodules import *
'Parameter count = 162,518,834'

'Parameter count = 162,518,834'

In [4]:
from models import *
import matplotlib.pyplot as plt

In [5]:
# import png
import numpy as np
import matplotlib.colors as cl
import matplotlib.pyplot as plt
from PIL import Image


UNKNOWN_FLOW_THRESH = 1e7
SMALLFLOW = 0.0
LARGEFLOW = 1e8

"""
=============
Flow Section
=============
"""


def show_flow(filename):
    """
    visualize optical flow map using matplotlib
    :param filename: optical flow file
    :return: None
    """
    flow = read_flow(filename)
    img = flow_to_image(flow)
    plt.imshow(img)
    plt.show()


def visualize_flow(flow, mode='Y'):
    """
    this function visualize the input flow
    :param flow: input flow in array
    :param mode: choose which color mode to visualize the flow (Y: Ccbcr, RGB: RGB color)
    :return: None
    """
    if mode == 'Y':
        # Ccbcr color wheel
        img = flow_to_image(flow)
        plt.imshow(img)
        plt.show()
    elif mode == 'RGB':
        (h, w) = flow.shape[0:2]
        du = flow[:, :, 0]
        dv = flow[:, :, 1]
        valid = flow[:, :, 2]
        max_flow = max(np.max(du), np.max(dv))
        img = np.zeros((h, w, 3), dtype=np.float64)
        # angle layer
        img[:, :, 0] = np.arctan2(dv, du) / (2 * np.pi)
        # magnitude layer, normalized to 1
        img[:, :, 1] = np.sqrt(du * du + dv * dv) * 8 / max_flow
        # phase layer
        img[:, :, 2] = 8 - img[:, :, 1]
        # clip to [0,1]
        small_idx = img[:, :, 0:3] < 0
        large_idx = img[:, :, 0:3] > 1
        img[small_idx] = 0
        img[large_idx] = 1
        # convert to rgb
        img = cl.hsv_to_rgb(img)
        # remove invalid point
        img[:, :, 0] = img[:, :, 0] * valid
        img[:, :, 1] = img[:, :, 1] * valid
        img[:, :, 2] = img[:, :, 2] * valid
        # show
        plt.imshow(img)
        plt.show()

    return None


def read_flow(filename):
    """
    read optical flow from Middlebury .flo file
    :param filename: name of the flow file
    :return: optical flow data in matrix
    """
    f = open(filename, 'rb')
    magic = np.fromfile(f, np.float32, count=1)
    data2d = None

    if 202021.25 != magic:
        print('Magic number incorrect. Invalid .flo file')
    else:
        w = np.fromfile(f, np.int32, count=1)
        h = np.fromfile(f, np.int32, count=1)
        print("Reading %d x %d flo file" % (h, w))
        data2d = np.fromfile(f, np.float32, count=2 * w * h)
        # reshape data into 3D array (columns, rows, channels)
        data2d = np.resize(data2d, (h[0], w[0], 2))
    f.close()
    return data2d


# def read_flow_png(flow_file):
#     """
#     Read optical flow from KITTI .png file
#     :param flow_file: name of the flow file
#     :return: optical flow data in matrix
#     """
#     flow_object = png.Reader(filename=flow_file)
#     flow_direct = flow_object.asDirect()
#     flow_data = list(flow_direct[2])
#     (w, h) = flow_direct[3]['size']
#     flow = np.zeros((h, w, 3), dtype=np.float64)
#     for i in range(len(flow_data)):
#         flow[i, :, 0] = flow_data[i][0::3]
#         flow[i, :, 1] = flow_data[i][1::3]
#         flow[i, :, 2] = flow_data[i][2::3]

#     invalid_idx = (flow[:, :, 2] == 0)
#     flow[:, :, 0:2] = (flow[:, :, 0:2] - 2 ** 15) / 64.0
#     flow[invalid_idx, 0] = 0
#     flow[invalid_idx, 1] = 0
#     return flow


def write_flow(flow, filename):
    """
    write optical flow in Middlebury .flo format
    :param flow: optical flow map
    :param filename: optical flow file path to be saved
    :return: None
    """
    f = open(filename, 'wb')
    magic = np.array([202021.25], dtype=np.float32)
    (height, width) = flow.shape[0:2]
    w = np.array([width], dtype=np.int32)
    h = np.array([height], dtype=np.int32)
    magic.tofile(f)
    w.tofile(f)
    h.tofile(f)
    flow.tofile(f)
    f.close()


def segment_flow(flow):
    h = flow.shape[0]
    w = flow.shape[1]
    u = flow[:, :, 0]
    v = flow[:, :, 1]

    idx = ((abs(u) > LARGEFLOW) | (abs(v) > LARGEFLOW))
    idx2 = (abs(u) == SMALLFLOW)
    class0 = (v == 0) & (u == 0)
    u[idx2] = 0.00001
    tan_value = v / u

    class1 = (tan_value < 1) & (tan_value >= 0) & (u > 0) & (v >= 0)
    class2 = (tan_value >= 1) & (u >= 0) & (v >= 0)
    class3 = (tan_value < -1) & (u <= 0) & (v >= 0)
    class4 = (tan_value < 0) & (tan_value >= -1) & (u < 0) & (v >= 0)
    class8 = (tan_value >= -1) & (tan_value < 0) & (u > 0) & (v <= 0)
    class7 = (tan_value < -1) & (u >= 0) & (v <= 0)
    class6 = (tan_value >= 1) & (u <= 0) & (v <= 0)
    class5 = (tan_value >= 0) & (tan_value < 1) & (u < 0) & (v <= 0)

    seg = np.zeros((h, w))

    seg[class1] = 1
    seg[class2] = 2
    seg[class3] = 3
    seg[class4] = 4
    seg[class5] = 5
    seg[class6] = 6
    seg[class7] = 7
    seg[class8] = 8
    seg[class0] = 0
    seg[idx] = 0

    return seg


def flow_error(tu, tv, u, v):
    """
    Calculate average end point error
    :param tu: ground-truth horizontal flow map
    :param tv: ground-truth vertical flow map
    :param u:  estimated horizontal flow map
    :param v:  estimated vertical flow map
    :return: End point error of the estimated flow
    """
    smallflow = 0.0
    '''
    stu = tu[bord+1:end-bord,bord+1:end-bord]
    stv = tv[bord+1:end-bord,bord+1:end-bord]
    su = u[bord+1:end-bord,bord+1:end-bord]
    sv = v[bord+1:end-bord,bord+1:end-bord]
    '''
    stu = tu[:]
    stv = tv[:]
    su = u[:]
    sv = v[:]

    idxUnknow = (abs(stu) > UNKNOWN_FLOW_THRESH) | (abs(stv) > UNKNOWN_FLOW_THRESH)
    stu[idxUnknow] = 0
    stv[idxUnknow] = 0
    su[idxUnknow] = 0
    sv[idxUnknow] = 0

    ind2 = [(np.absolute(stu) > smallflow) | (np.absolute(stv) > smallflow)]
    index_su = su[ind2]
    index_sv = sv[ind2]
    an = 1.0 / np.sqrt(index_su ** 2 + index_sv ** 2 + 1)
    un = index_su * an
    vn = index_sv * an

    index_stu = stu[ind2]
    index_stv = stv[ind2]
    tn = 1.0 / np.sqrt(index_stu ** 2 + index_stv ** 2 + 1)
    tun = index_stu * tn
    tvn = index_stv * tn

    '''
    angle = un * tun + vn * tvn + (an * tn)
    index = [angle == 1.0]
    angle[index] = 0.999
    ang = np.arccos(angle)
    mang = np.mean(ang)
    mang = mang * 180 / np.pi
    '''

    epe = np.sqrt((stu - su) ** 2 + (stv - sv) ** 2)
    epe = epe[ind2]
    mepe = np.mean(epe)
    return mepe


def flow_to_image(flow):
    """
    Convert flow into middlebury color code image
    :param flow: optical flow map
    :return: optical flow image in middlebury color
    """
    u = flow[:, :, 0]
    v = flow[:, :, 1]

    maxu = -999.
    maxv = -999.
    minu = 999.
    minv = 999.

    idxUnknow = (abs(u) > UNKNOWN_FLOW_THRESH) | (abs(v) > UNKNOWN_FLOW_THRESH)
    u[idxUnknow] = 0
    v[idxUnknow] = 0

    maxu = max(maxu, np.max(u))
    minu = min(minu, np.min(u))

    maxv = max(maxv, np.max(v))
    minv = min(minv, np.min(v))

    rad = np.sqrt(u ** 2 + v ** 2)
    maxrad = max(-1, np.max(rad))

    print("max flow: %.4f\nflow range:\nu = %.3f .. %.3f\nv = %.3f .. %.3f" % (maxrad, minu,maxu, minv, maxv))

    u = u/(maxrad + np.finfo(float).eps)
    v = v/(maxrad + np.finfo(float).eps)

    img = compute_color(u, v)

    idx = np.repeat(idxUnknow[:, :, np.newaxis], 3, axis=2)
    img[idx] = 0

    return np.uint8(img)


def evaluate_flow_file(gt, pred):
    """
    evaluate the estimated optical flow end point error according to ground truth provided
    :param gt: ground truth file path
    :param pred: estimated optical flow file path
    :return: end point error, float32
    """
    # Read flow files and calculate the errors
    gt_flow = read_flow(gt)        # ground truth flow
    eva_flow = read_flow(pred)     # predicted flow
    # Calculate errors
    average_pe = flow_error(gt_flow[:, :, 0], gt_flow[:, :, 1], eva_flow[:, :, 0], eva_flow[:, :, 1])
    return average_pe


def evaluate_flow(gt_flow, pred_flow):
    """
    gt: ground-truth flow
    pred: estimated flow
    """
    average_pe = flow_error(gt_flow[:, :, 0], gt_flow[:, :, 1], pred_flow[:, :, 0], pred_flow[:, :, 1])
    return average_pe


"""
==============
Disparity Section
==============
"""

# 
# def read_disp_png(file_name):
#     """
#     Read optical flow from KITTI .png file
#     :param file_name: name of the flow file
#     :return: optical flow data in matrix
#     """
#     image_object = png.Reader(filename=file_name)
#     image_direct = image_object.asDirect()
#     image_data = list(image_direct[2])
#     (w, h) = image_direct[3]['size']
#     channel = len(image_data[0]) / w
#     flow = np.zeros((h, w, channel), dtype=np.uint16)
#     for i in range(len(image_data)):
#         for j in range(channel):
#             flow[i, :, j] = image_data[i][j::channel]
#     return flow[:, :, 0] / 256


def disp_to_flowfile(disp, filename):
    """
    Read KITTI disparity file in png format
    :param disp: disparity matrix
    :param filename: the flow file name to save
    :return: None
    """
    f = open(filename, 'wb')
    magic = np.array([202021.25], dtype=np.float32)
    (height, width) = disp.shape[0:2]
    w = np.array([width], dtype=np.int32)
    h = np.array([height], dtype=np.int32)
    empty_map = np.zeros((height, width), dtype=np.float32)
    data = np.dstack((disp, empty_map))
    magic.tofile(f)
    w.tofile(f)
    h.tofile(f)
    data.tofile(f)
    f.close()


"""
==============
Image Section
==============
"""


def read_image(filename):
    """
    Read normal image of any format
    :param filename: name of the image file
    :return: image data in matrix uint8 type
    """
    img = Image.open(filename)
    im = np.array(img)
    return im


def warp_image(im, flow):
    """
    Use optical flow to warp image to the next
    :param im: image to warp
    :param flow: optical flow
    :return: warped image
    """
    from scipy import interpolate
    image_height = im.shape[0]
    image_width = im.shape[1]
    flow_height = flow.shape[0]
    flow_width = flow.shape[1]
    n = image_height * image_width
    (iy, ix) = np.mgrid[0:image_height, 0:image_width]
    (fy, fx) = np.mgrid[0:flow_height, 0:flow_width]
    fx += flow[:,:,0]
    fy += flow[:,:,1]
    mask = np.logical_or(fx <0 , fx > flow_width)
    mask = np.logical_or(mask, fy < 0)
    mask = np.logical_or(mask, fy > flow_height)
    fx = np.minimum(np.maximum(fx, 0), flow_width)
    fy = np.minimum(np.maximum(fy, 0), flow_height)
    points = np.concatenate((ix.reshape(n,1), iy.reshape(n,1)), axis=1)
    xi = np.concatenate((fx.reshape(n, 1), fy.reshape(n,1)), axis=1)
    warp = np.zeros((image_height, image_width, im.shape[2]))
    for i in range(im.shape[2]):
        channel = im[:, :, i]
        plt.imshow(channel, cmap='gray')
        values = channel.reshape(n, 1)
        new_channel = interpolate.griddata(points, values, xi, method='cubic')
        new_channel = np.reshape(new_channel, [flow_height, flow_width])
        new_channel[mask] = 1
        warp[:, :, i] = new_channel.astype(np.uint8)

    return warp.astype(np.uint8)


"""
==============
Others
==============
"""

def scale_image(image, new_range):
    """
    Linearly scale the image into desired range
    :param image: input image
    :param new_range: the new range to be aligned
    :return: image normalized in new range
    """
    min_val = np.min(image).astype(np.float32)
    max_val = np.max(image).astype(np.float32)
    min_val_new = np.array(min(new_range), dtype=np.float32)
    max_val_new = np.array(max(new_range), dtype=np.float32)
    scaled_image = (image - min_val) / (max_val - min_val) * (max_val_new - min_val_new) + min_val_new
    return scaled_image.astype(np.uint8)


def compute_color(u, v):
    """
    compute optical flow color map
    :param u: optical flow horizontal map
    :param v: optical flow vertical map
    :return: optical flow in color code
    """
    [h, w] = u.shape
    img = np.zeros([h, w, 3])
    nanIdx = np.isnan(u) | np.isnan(v)
    u[nanIdx] = 0
    v[nanIdx] = 0

    colorwheel = make_color_wheel()
    ncols = np.size(colorwheel, 0)

    rad = np.sqrt(u**2+v**2)

    a = np.arctan2(-v, -u) / np.pi

    fk = (a+1) / 2 * (ncols - 1) + 1

    k0 = np.floor(fk).astype(int)

    k1 = k0 + 1
    k1[k1 == ncols+1] = 1
    f = fk - k0

    for i in range(0, np.size(colorwheel,1)):
        tmp = colorwheel[:, i]
        col0 = tmp[k0-1] / 255
        col1 = tmp[k1-1] / 255
        col = (1-f) * col0 + f * col1

        idx = rad <= 1
        col[idx] = 1-rad[idx]*(1-col[idx])
        notidx = np.logical_not(idx)

        col[notidx] *= 0.75
        img[:, :, i] = np.uint8(np.floor(255 * col*(1-nanIdx)))

    return img


def make_color_wheel():
    """
    Generate color wheel according Middlebury color code
    :return: Color wheel
    """
    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR

    colorwheel = np.zeros([ncols, 3])

    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.transpose(np.floor(255*np.arange(0, RY) / RY))
    col += RY

    # YG
    colorwheel[col:col+YG, 0] = 255 - np.transpose(np.floor(255*np.arange(0, YG) / YG))
    colorwheel[col:col+YG, 1] = 255
    col += YG

    # GC
    colorwheel[col:col+GC, 1] = 255
    colorwheel[col:col+GC, 2] = np.transpose(np.floor(255*np.arange(0, GC) / GC))
    col += GC

    # CB
    colorwheel[col:col+CB, 1] = 255 - np.transpose(np.floor(255*np.arange(0, CB) / CB))
    colorwheel[col:col+CB, 2] = 255
    col += CB

    # BM
    colorwheel[col:col+BM, 2] = 255
    colorwheel[col:col+BM, 0] = np.transpose(np.floor(255*np.arange(0, BM) / BM))
    col += + BM

    # MR
    colorwheel[col:col+MR, 2] = 255 - np.transpose(np.floor(255 * np.arange(0, MR) / MR))
    colorwheel[col:col+MR, 0] = 255

    return colorwheel

def plot_feature_dim(lstm_input):
    fig, axs = plt.subplots(4,3,figsize=(15,15))

    for i in range(4):
        for j in range(3):
            if (i == 3 and j == 2):
                break

            x = np.arange(0,80000)
            axs[i,j].plot(x,lstm_input[:,i*3+j])
            axs[i,j].set_title(f'Feature {i*3+j+1}')

# Sample the source video

In [ ]:
cap = cv2.VideoCapture('data/caolin_foot_trial1-4.mp4')
name = 'caolin'

framerate = cap.get(5)
total_image = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f'The oringinal framerate is {cap.get(5)} with frame resolution of: {cap.get(3), cap.get(4)}')
print(f'The total number of frame in this video is {total_image}')

In [ ]:
count = 1
while(cap.isOpened()):
    frameID = cap.get(1) # get the current frame number
    ret, frame = cap.read()
    
    if(ret != True):
        print(f'We\'ve gotten {int(frameID/5)+1} frames from this video.')
        break
    # Take at 5Hz frequency which is framerate/5
    if (frameID % int(framerate/5) == 0):
        frame = frame[64:, 162:610 , :]
        filename = 'Train_1/'+ name + "_frame%04d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()

# Building Dataset

In [6]:
print("Directory Path:", Path().absolute())  

Directory Path: /home/projectx/Documents/other people's repo/flownet2-pytorch


In [7]:
class MyDataset:
    
    def __init__(self, num_train_img = 3400, base_dir = Path().absolute(), sequence = Path('Train_1/')):  # base_dir(image) and sequence(lstm) are directories
        self.base_dir = base_dir
        self.sequence = sequence
        self.base_path_img = os.path.join(self.base_dir, self.sequence)
        
#         self.base_dir + self.sequence
#         os.path.join("source_data", "text_files")
        
        self.image_files = os.listdir(self.base_path_img)
        self.image_files.sort()
        self.image_files = self.image_files[0:num_train_img]
        
        # normalization for lstm data
        self.train_scaler = preprocessing.StandardScaler()
        
        ## Omega.7 and load cells
        self.input_lstm = self.read_OMEGA7_LC()
        
        
        self.imu_seq_len = 20
    
    def read_OMEGA7_LC(self, path='data/result_1.csv'):
        # read csv data
        df = pd.read_csv(path,header = None)
        df = df[:874300]
        
        # take moving average of every 10 data points
        new_df = df.groupby(df.index//10).mean()
        array_input = np.array(new_df)
        
        # normalization
        array_input_scaled = self.train_scaler.fit_transform(array_input)
        
        input_lstm  = Variable(torch.from_numpy(array_input_scaled).type(torch.FloatTensor))
        input_lstm = input_lstm[:80000,:]
        
        
        
        # reshape to (num_dataset, sequence_length, feature_size)
        input_lstm = input_lstm.view(-1,20,11)
        
        return np.array(input_lstm)
    
    
    def get_input_lstm(self):
        return self.input_lstm
    
    def __len__(self):
        return len(self.image_files)
    
    def load_img_bat(self, idx, batch):
        batch_x = []
        batch_input_lstm = []
        for i in range(batch):
            x_data_np_1 = np.array(Image.open(os.path.join(self.base_path_img, Path(self.image_files[idx + i]))))
            x_data_np_2 = np.array(Image.open(os.path.join(self.base_path_img, Path(self.image_files[idx+1 + i]))))
            x_data_np_1 = x_data_np_1.reshape(3,512,448)
            x_data_np_2 = x_data_np_2.reshape(3,512,448)

#             ## 3 channels
#             x_data_np_1 = np.array([x_data_np_1, x_data_np_1, x_data_np_1])
#             x_data_np_2 = np.array([x_data_np_2, x_data_np_2, x_data_np_2])

            X = np.array([x_data_np_1, x_data_np_2])
            batch_x.append(X)
        
#           self.input_lstm of size: (num_dataset, sequence_length, feature_size)
            tmp = np.array(self.input_lstm[idx + i])
            batch_input_lstm.append(tmp)
            
        
        batch_x = np.array(batch_x).transpose( 0, 2, 1, 3,4)
        batch_input_lstm = np.array(batch_input_lstm)
        
        X = Variable(torch.from_numpy(batch_x).type(torch.FloatTensor).cuda())    
        X2 = Variable(torch.from_numpy(batch_input_lstm).type(torch.FloatTensor).cuda())    
        
        Y = X2[:,:,-3:]
        
     
        return X, X2 , Y.mean(1)

In [ ]:
dataset = MyDataset()
X, X2 , Y = dataset.load_img_bat(0,1)
X.size(), X2.size() , Y.size()

In [ ]:
lstm_data = dataset.get_input_lstm()
# Though there are 4000, we will only call first 3400, constrained by the length of 'num_train_img'
lstm_data.shape

In [ ]:
lstm_input = dataset.read_OMEGA7_LC()
lstm_input = lstm_input.reshape(4000*20, -1)
plot_feature_dim(lstm_input)
lstm_input.shape

# Hybrid Network

In [8]:
batch_size = 1
class Surgical_VINet(nn.Module):
    def __init__(self):
        super(Surgical_VINet, self).__init__()
        self.rnn = nn.LSTM(
            input_size=28792 ,#49152,#24576, 
            hidden_size=256,#64, 
            num_layers=2,
            batch_first=True)
        self.rnn.cuda()
        
        self.rnnIMU = nn.LSTM(
            input_size=11, 
            hidden_size=6,
            num_layers=2,
            batch_first=True)
        self.rnnIMU.cuda()
        
        self.linear1 = nn.Linear(256, 128)
        self.linear2 = nn.Linear(128, 3)
        #self.linear3 = nn.Linear(128, 6)
        self.linear1.cuda()
        self.linear2.cuda()
        #self.linear3.cuda()
        
        # load checkpoint state from NVIDIA training
           
        print('....creating model....')
        self.flownet_c = FlowNet2(args).cuda()
        
        print('....loading weights....')
        self.state_dict = torch.load("FlowNet2_checkpoint.pth.tar")
        self.flownet_c.load_state_dict(self.state_dict["state_dict"])
        self.flownet_c.cuda()
        
        print('....model created....')
        for param in self.flownet_c.parameters():
            param.requires_grad = False
        print('....model weights freezed....')
        
        
        

    def forward(self, image, imu):
        
        c_out = self.flownet_c(image) #[2, 128, 112] # [1, 2, 8, 7]
#         print('c_out', c_out.shape)
        
        ## Input2: Feed IMU records to LSTM
        imu_out, (imu_n, imu_c) = self.rnnIMU(imu)
        # to match Vision output shape
        imu_out = imu_out.view(batch_size,1,-1)   # (batch_size, 1, total_hidden_size) [1, 1, 120]
        
#         print('imu_out', imu_out.shape)
        
        
        ## Combine the output of input1 and 2 and feed it to LSTM
        #r_in = c_out.view(batch_size, timesteps, -1)
        r_in = c_out.view(batch_size, 1, -1) # [1,1,28672] # [1,1,112]
#         print('r_in', r_in.shape)
        

        cat_out = torch.cat((r_in, imu_out), 2)#[1 1 28792]
#         print(cat_out.shape)
        
        
        r_out, (h_n, h_c) = self.rnn(cat_out)  # (1, 1, 256)
#         print('r_out', r_out.shape)
        l_out1 = self.linear1(r_out[:,-1,:])
        l_out2 = self.linear2(l_out1)
        
#         print('r_ol_out2ut', l_out2.shape)
        #l_out3 = self.linear3(l_out2)

        return l_out2

# Training

In [9]:
def train():
    # hyper-parameters
    epoch = 2
    batch = 1
    lr = 0.001
    
    # creat model and dataset
    model = Surgical_VINet()
    mydataset = MyDataset()
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
#     optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    # TensorBoard
    writer = SummaryWriter()
 
    model.train()
    
    mydataset = MyDataset()
    
#     criterion = nn.MSELoss()
    criterion  = nn.L1Loss(size_average=False)
    
    start = 5
    end = len(mydataset) - batch
    batch_num = (end - start)
    startT = time.time()
    
    with tools.TimerBlock("Start training") as block :
        for k in range(epoch):
            for i in range(start, end):
                data, data_lstm, target = mydataset.load_img_bat(i, batch)
#                 data, data_lstm, target = data.cuda(), data_lstm.cuda(), target.cuda()
                
                optimizer.zero_grad()
                
                # forward pass
                output = model(data, data_lstm)
                
                # compute loss
                loss = criterion(output, target)
                
                # backward pass
                loss.backward()
#                 torch.nn.utils.clip_grad_value_(model.parameters(), 1.0)
                optimizer.step()
                
                avgTime = block.avg()
                remainingTime = int((batch_num*epoch -  (i + batch_num*k)) * avgTime)
                rTime_str = "{:02d}:{:02d}:{:02d}".format(int(remainingTime/60//60), 
                                                          int(remainingTime//60%60), 
                                                          int(remainingTime%60))
                block.log(f'Train Epoch: {k}\t[{i}/{batch_num} ({(100.*(i + batch_num*k)):.0f}%)]\tLoss: {loss.data:.6f}, TimeAvg: {avgTime:.4f}, Remaining: {rTime_str}')
                          
                writer.add_scalar('loss', loss.data, k*batch_num + i)
                

                
                
                
            check_str = 'checkpoint_{}.pt'.format(k)
            if (k+1)%5 == 0:
                torch.save(model.state_dict(), check_str)
    
    torch.save(model.state_dict(), 'test_network_trial_1.pt')
    writer.export_scalars_to_json("./summary_writer.json")
    writer.close()

In [10]:
train()

....creating model....
....loading weights....
....model created....
....model weights freezed....


/home/projectx/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/home/projectx/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/home/projectx/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/home/projectx/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of n

Start training
  [0.477s] Train Epoch: 0	[5/3394 (500%)]	Loss: 2.145501, TimeAvg: 0.4490, Remaining: 00:50:45
  [0.838s] Train Epoch: 0	[6/3394 (600%)]	Loss: 1.621291, TimeAvg: 0.8294, Remaining: 01:33:45
  [1.210s] Train Epoch: 0	[7/3394 (700%)]	Loss: 1.328026, TimeAvg: 0.6010, Remaining: 01:07:55
  [1.579s] Train Epoch: 0	[8/3394 (800%)]	Loss: 1.126436, TimeAvg: 0.5232, Remaining: 00:59:07
  [1.942s] Train Epoch: 0	[9/3394 (900%)]	Loss: 1.004346, TimeAvg: 0.4834, Remaining: 00:54:37
  [2.286s] Train Epoch: 0	[10/3394 (1000%)]	Loss: 0.761315, TimeAvg: 0.4554, Remaining: 00:51:26
  [2.631s] Train Epoch: 0	[11/3394 (1100%)]	Loss: 0.398882, TimeAvg: 0.4361, Remaining: 00:49:15
  [3.123s] Train Epoch: 0	[12/3394 (1200%)]	Loss: 0.125067, TimeAvg: 0.4441, Remaining: 00:50:09
  [3.462s] Train Epoch: 0	[13/3394 (1300%)]	Loss: 0.472103, TimeAvg: 0.4311, Remaining: 00:48:40
  [3.796s] Train Epoch: 0	[14/3394 (1400%)]	Loss: 0.586899, TimeAvg: 0.4206, Remaining: 00:47:29
  [4.131s] Train Epoch: 0

  [30.113s] Train Epoch: 0	[90/3394 (9000%)]	Loss: 1.882463, TimeAvg: 0.3541, Remaining: 00:39:31
  [30.530s] Train Epoch: 0	[91/3394 (9100%)]	Loss: 1.757292, TimeAvg: 0.3549, Remaining: 00:39:36
  [30.901s] Train Epoch: 0	[92/3394 (9200%)]	Loss: 1.647303, TimeAvg: 0.3550, Remaining: 00:39:37
  [31.240s] Train Epoch: 0	[93/3394 (9300%)]	Loss: 1.498556, TimeAvg: 0.3549, Remaining: 00:39:35
  [31.572s] Train Epoch: 0	[94/3394 (9400%)]	Loss: 1.311124, TimeAvg: 0.3546, Remaining: 00:39:33
  [31.910s] Train Epoch: 0	[95/3394 (9500%)]	Loss: 2.440348, TimeAvg: 0.3544, Remaining: 00:39:32
  [32.255s] Train Epoch: 0	[96/3394 (9600%)]	Loss: 0.982241, TimeAvg: 0.3544, Remaining: 00:39:31
  [32.634s] Train Epoch: 0	[97/3394 (9700%)]	Loss: 1.433155, TimeAvg: 0.3546, Remaining: 00:39:32
  [32.993s] Train Epoch: 0	[98/3394 (9800%)]	Loss: 0.213707, TimeAvg: 0.3547, Remaining: 00:39:32
  [33.327s] Train Epoch: 0	[99/3394 (9900%)]	Loss: 0.237679, TimeAvg: 0.3544, Remaining: 00:39:30
  [33.660s] Train Ep

  [1.008m] Train Epoch: 0	[175/3394 (17500%)]	Loss: 1.398062, TimeAvg: 0.3555, Remaining: 00:39:11
  [1.014m] Train Epoch: 0	[176/3394 (17600%)]	Loss: 1.357893, TimeAvg: 0.3557, Remaining: 00:39:12
  [1.020m] Train Epoch: 0	[177/3394 (17700%)]	Loss: 1.352181, TimeAvg: 0.3559, Remaining: 00:39:12
  [1.026m] Train Epoch: 0	[178/3394 (17800%)]	Loss: 1.328584, TimeAvg: 0.3559, Remaining: 00:39:12
  [1.033m] Train Epoch: 0	[179/3394 (17900%)]	Loss: 1.315452, TimeAvg: 0.3561, Remaining: 00:39:13
  [1.039m] Train Epoch: 0	[180/3394 (18000%)]	Loss: 1.256762, TimeAvg: 0.3562, Remaining: 00:39:13
  [1.045m] Train Epoch: 0	[181/3394 (18100%)]	Loss: 1.210463, TimeAvg: 0.3563, Remaining: 00:39:14
  [1.051m] Train Epoch: 0	[182/3394 (18200%)]	Loss: 1.173370, TimeAvg: 0.3562, Remaining: 00:39:13
  [1.058m] Train Epoch: 0	[183/3394 (18300%)]	Loss: 1.138433, TimeAvg: 0.3565, Remaining: 00:39:14
  [1.064m] Train Epoch: 0	[184/3394 (18400%)]	Loss: 1.095427, TimeAvg: 0.3566, Remaining: 00:39:14
  [1.070m]

KeyboardInterrupt: 

# Create Model

In [ ]:
print('creating model.................')
net = FlowNet2(args).cuda()
print('loading weight.................')
state_dict = torch.load("FlowNet2_checkpoint.pth.tar")
net.load_state_dict(state_dict["state_dict"])

In [ ]:
# pim1 = read_gen("/home/projectx/Downloads/training/clean/ambush_5/frame_0037.png")
# pim2 = read_gen("/home/projectx/Downloads/training/clean/ambush_5/frame_0038.png")
pim1 = read_gen("/home/projectx/Documents/GitHub repos/VINET_modification/Train_1/caolin_frame0822.jpg")
pim2 = read_gen("/home/projectx/Documents/GitHub repos/VINET_modification/Train_1/caolin_frame0824.jpg")

images = [pim1[:,:], pim2[:,:]]
images = np.array(images).transpose(3, 0, 1, 2)
im = torch.from_numpy(images.astype(np.float32)).unsqueeze(0).cuda()
print(im.size())

plt.imshow(pim1)
plt.show()

plt.imshow(pim2)
plt.show()

result = net(im).squeeze()

result.size()

In [ ]:
flow = result.data.cpu().numpy().transpose(1, 2, 0)
# flow.shape

image = flow_to_image(flow)

plt.imshow(image)
plt.show()

In [ ]:
image.shape

In [ ]:
plt.imshow(image)
plt.show()

# ------------------------------------------------

In [ ]:
model = Surgical_VINet()

In [ ]:
result = model.flownet_c(im).squeeze()
result.size()

In [ ]:
output = model(X,X2)
(output.size(), Y.size())

In [ ]:
criterion = nn.MSELoss()

loss = criterion(output, Y)
loss

In [ ]:
criterion = nn.L1Loss(size_average=False)

loss = criterion(output, Y)
loss